In [70]:
#create csv file with scores of 100 students in 4 course- RUN ONLY ONCE
def create_students_df():    
    #create empty df that will contains the students' scores in all courses
    main_df=pd.DataFrame(columns=["student","course","score"])
    
    #create student's list
    fake = Faker() # use faker in order to generate random students' names
    students=[]
    while len(students)<100:
        name=fake.first_name()
        if name not in students:
            students.append(name)
        else:
            continue

    # list of courses and its mean and std
    primary_courses={ 
        "Math":[75,12], #mean=75, std=12
        "Art": [80,10],
        "Physics": [77,11],
        "Sport":[80,15]}
    
    #create score for each student in each course
    for course,stats in primary_courses.items():    
        mean,std=stats
        scores = np.random.normal(mean, std,size=(100,)) #create 100 scores that distributed normally
        scores=[max(0, min(int(score), 100)) for score in scores] #convert to int and validates 0<= x >=100
        data_to_course_df = {"student": students, "course": [course] * len(students), "score": scores}
        course_df=pd.DataFrame(data_to_course_df)
        main_df=pd.concat([main_df,course_df],ignore_index=True)

    main_df.to_csv('student_list.csv',index=False)     # TODO: use an external .csv / .yml file to store it. DONE

In [109]:
# TODO: this entire gloval vars thing is a mess, fix it. generally- 
#don't use global vars, just a dataframe or whatever that you pass on between functions
#ido- ask tal what is the difference between main_df and dictationary
def add_score(student_name=None,course_name=None):
    global main_df,global_vars
    while True:
        if student_name is None:
            student_name=input(f"Student name: (student for example:{student[random.randint(0, len(student)-1)]})") .title().strip()
            global_vars['student_name']=student_name
        if  not validation(student_name,student):
            validation_failed(student_name,student)
            break
        else:
            if course_name is None:
                course_name=input(f'course: (course for example:{course[random.randint(0,len(course)-1)]}) ').title().strip()
                global_vars['course_name']=course_name
            if  not validation(course_name,course):
                validation_failed(course_name,course)
                break
            else:
                if score_duplication_validation(student_name,course_name):
                    score=int(input("score: ").strip())
                    global_vars['score']=score
                    if score_validation(score):
                        new_score=pd.Series({'student':global_vars['student_name'],'course':global_vars['course_name'],'score':global_vars['score']})
                        main_df=pd.concat([main_df,new_score.to_frame().T],ignore_index=True)
                        print(f"we updated that {global_vars['student_name']} gets {global_vars['score']} in {global_vars['course_name']}")
                else:
                    update_score=input(f"Do you want to update {global_vars['student_name']}'s score? (yes/no)")
                    if update_score== 'yes':
                        correct_score(student_name=student_name,course_name=course_name)
        next_action= input("Do you want to add another score? (yes/no): ").strip().lower()
        if next_action != 'yes':
            break
        else:
            student_name=None
            course_name=None
            continue

In [110]:
# TODO: add new student/ add new subject look almost the same - code duplication at large-DONE
# used to add new student or course
def add_new(column,new_value=None):
    global main_df,global_vars
    while True:
        if new_value is None:
            new_value=input(f"New {column}'s Name:").title().strip()

        if new_value in global_vars[column].values:
            print(f"There is already {new_value} in the {column}'s list")
        else:
            try:
                if new_value.isalpha():#the only validation for student name is that it only contains letters 
                    # TODO: the hell is this - annotation abuse  name, name= is horrible practice. DONE
                    # TODO: Rename subject to attending_subject or something DONE (changed to course)
                    value_to_add=pd.Series(new_value,name=column)
                    if column=='student':
                        global_vars['student']=pd.concat([global_vars['student'],value_to_add],ignore_index=True)
                    else:
                        global_vars['course']=pd.concat([global_vars['course'],value_to_add],ignore_index=True)
                    print(f"{new_value} is added to the {column}'s list")
                else:
                    raise ValueError(f"{column}'s Name can only contain letters")
            except ValueError as e:
                print(e)
        next_action= input(f"Do you want to add another {column}? (yes/no): ").strip().lower()
        new_value=None
        if next_action != 'yes':
            break

In [111]:
def correct_score(student_name=None,course_name=None):
    global main_df,global_vars
    while True:
        if student_name is None:
            student_name=input(f'Student name: (student for example:{student[random.randint(0,len(student)-1)]})').title().strip()
        
        #validate that the name exists in the student's list 
        if not validation(student_name,student):
            validation_failed(student_name,student)
            break
        else:
            global_vars['student_name']=student_name
            if course_name is None:
                # TODO: randint(0, 3) is not generic at all. use len(course) instead or something like that DONE
                course_name=input(f'course: (course for example:{course[random.randint(0,len(course)-1)]})').title().strip()
            
            #validate that the name exists in the course's list
            if not validation(course_name,course):
                validation_failed(course_name,course)
                break
            else:
                global_vars['course_name']=course_name 
                current_score=main_df[(main_df['student']==student_name)&(main_df['course']==course_name)]
                
                #check if the student already has score in this course
                # TODO: try to get read of the global vars - it's not a good practice + not intuitive
                if current_score.shape[0]>0:
                    print(f"{student_name} has score of {int(current_score['score'])} in {course_name}, what is the updated score?")
                    score=int(input("Enter new score ").strip())
                    global_vars['score']=score
                    if score_validation(score):
                        
                        #update the new score 
                        # TODO - why global_vars['student_name'] and not just student_name? DONE
                        main_df.loc[(main_df['student'] == student_name) & (main_df['course'] == course_name), 'score'] = global_vars['score']
                        print(f"we updated that {student_name} gets {score} in {course_name}")
                    next_action= input("Do you want to correct another student's score? (yes/no): ").strip().lower()
                    if next_action != 'yes':
                        break
                else:
                    next_action=input(f"{student_name} doesn't have a score in {course_name}, do you want to enter new score?").strip().lower()
                    if next_action != 'yes':
                        break
                    else:
                        add_score(student_name=student_name,course_name=course_name)
                        break

In [112]:
def validation_failed(name,column):
    print(f"""There isn't {name} in the {column.name}'s list
    What do you want to do:
    1) Add {name} as new {column.name}
    2) I want to correct the {column.name}'s name
    3) Return to main menu""")
    next_action=input("Enter the number of the desired action ")
    match next_action:
        case '1':
            # TODO: duplication of lines. you can do if validation(name, column.name) for instance
            #ido- related to the decision to make 2 different functions, one for students and one for courses
            if column.name=='student':
                add_new_student(name)
            else:
                add_new_course(name)
        case '2':
            # TODO: it's more common to use f-strings than .format. f"New {column.name}'s name: "DONE
            new_name=input(f"New {column.name}'s name: ").title().strip()
            if column.name=="student":
                global_vars['student_name']=new_name
                add_score(student_name=global_vars['student_name'])
            else:
                global_vars['course_name']=new_name
                add_score(student_name=global_vars['student_name'],course_name=global_vars['course_name'])
        case '3':
            return